In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 1. Alamat website target
url = "https://poskobanjir.dsdadki.web.id/"

# 2. Ambil konten dari website
headers = {'User-Agent': 'Mozilla/5.0'} # Agar tidak diblokir sebagai bot
response = requests.get(url, headers=headers)

if response.status_code == 200:
    html_content = response.text # Sekarang html_content sudah terdefinisi!
    print("✅ Berhasil mengambil data dari website.")
    
    # 3. Proses dengan BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Bidik tabel berdasarkan ID yang kamu berikan tadi
    table = soup.find('table', {'id': 'ContentPlaceHolder1_CtrlDataTinggiAir_GridListPintuAir_DXMainTable'})
    
    water_data = []
    rows = table.find_all('tr', id=lambda x: x and 'DXDataRow' in x)

    for row in rows:
        cols = row.find_all('td')
        
        # Ekstraksi data
        lokasi = cols[1].text.strip()
        sungai = cols[2].text.strip()
        tinggi_air = cols[3].find('span').text.strip()
        
        # Deteksi Tren (Naik/Turun) dari Icon
        img_src = cols[4].find('img')['src']
        tren = "Naik" if "up" in img_src else "Turun" if "down" in img_src else "Stabil"
        
        # Deteksi Status Siaga dari Icon
        status_src = cols[7].find('img')['src']
        status = "Normal"
        if "siaga1" in status_src: status = "Siaga 1"
        elif "siaga2" in status_src: status = "Siaga 2"
        elif "siaga3" in status_src: status = "Siaga 3"
        
        # Di dalam loop 'for row in rows:' yang tadi:
        tanggal = cols[5].find('span').text.strip() # Mengambil "22/01/2026"
        jam = cols[6].find('span').text.strip()     # Mengambil "13:40"

        water_data.append({
            'lokasi': lokasi,
            'sungai': sungai,
            'tma_cm': int(tinggi_air),
            'waktu': pd.to_datetime(f"{tanggal} {jam}", dayfirst=True),
            'tren': tren,
            'status': status
        })

    # 4. Tampilkan Hasil
    df_tma = pd.DataFrame(water_data)
    print(df_tma)
else:
    print(f"❌ Gagal mengambil data. Status code: {response.status_code}")

✅ Berhasil mengambil data dari website.
                             lokasi              sungai  tma_cm  \
0   Bendung. Cibalok - Gadog (Baru)            Ciliwung       0   
1              Bendung. Katulampa 2            Ciliwung      41   
2                      P.S. Depok 2            Ciliwung     119   
3           Kampung Rambutan (Baru)            Cipinang      60   
4                  P.A. Manggarai 2            Ciliwung     627   
5                P.S. Krukut Hulu 2              Krukut      54   
6               Pompa Cideng (Baru)      Cideng Siantar      54   
7                 P.A. Karet (Baru)  Banjir Kanal Barat     335   
8                 P.A. Marina Ancol                Laut     197   
9          P.A. Marina Ancol (Baru)                Kali     136   
10          Pompa Pasar Ikan (Baru)                Laut     187   
11         Rumah Pompa Pluit (Baru)         Waduk Pluit    -179   
12         P.S. Pesanggrahan (Baru)        Pesanggrahan      84   
13           P.S. Angk

In [ ]:
df_tma

In [ ]:
import pandas as pd
import time
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# --- KONFIGURASI ---
START_DATE = "2024-01-01"  # Mulai dari tanggal ini
END_DATE = "2024-01-05"    # Sampai tanggal ini (Coba rentang pendek dulu, misal 5 hari)
OUTPUT_FILE = "../data/data_tma_bpbd_2024.csv"

def scrape_history():
    # Setup Browser
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless") # Hapus komentar ini jika ingin berjalan di background
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    url = "https://bpbd.jakarta.go.id/waterlevel"
    driver.get(url)
    time.sleep(3) # Tunggu loading awal

    all_data = []
    
    # Generate list tanggal
    start = datetime.strptime(START_DATE, "%Y-%m-%d")
    end = datetime.strptime(END_DATE, "%Y-%m-%d")
    date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days + 1)]

    print(f"🔄 Memulai scraping dari {START_DATE} sampai {END_DATE}...")

    for date_obj in date_generated:
        target_date_str = date_obj.strftime("%Y-%m-%d")
        print(f"⏳ Mengambil data tanggal: {target_date_str}")
        
        try:
            # 1. Cari elemen input tanggal (ID: datetma)
            # Kita gunakan JavaScript untuk mengisi nilainya agar format tidak error
            driver.execute_script(f"document.getElementById('datetma').value = '{target_date_str}'")
            
            # 2. Klik tombol Filter (Class: mybutton1)
            filter_btn = driver.find_element(By.CLASS_NAME, "mybutton1")
            filter_btn.click()
            
            # 3. Tunggu halaman reload dan tabel muncul
            time.sleep(4) # Beri jeda agak lama agar server merespon
            
            # 4. Ambil HTML tabel dengan BeautifulSoup (lebih cepat parsing-nya)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Cari tabel (biasanya tabel di BPBD ada class 'table' atau di dalam div responsif)
            # Kita cari tabel yang memiliki header "Pintu Air"
            table = soup.find('table')
            
            if table:
                rows = table.find_all('tr')
                
                # Baris pertama biasanya header jam (11:00, 10:00, dst)
                # Kita perlu logika khusus karena struktur tabelnya matriks (Baris=Lokasi, Kolom=Jam)
                
                # Ambil header jam dari baris header (biasanya baris ke-2 di BPBD)
                header_row = rows[1].find_all('td') if len(rows) > 1 else []
                jam_list = [col.text.strip() for col in header_row if col.text.strip() != 'Pintu Air']
                
                # Loop baris data (mulai dari baris ke-3)
                for row in rows[2:]:
                    cols = row.find_all('td')
                    if not cols: continue
                    
                    nama_pintu = cols[0].text.strip()
                    
                    # Loop setiap kolom jam
                    for i, col in enumerate(cols[1:]):
                        if i < len(jam_list):
                            jam = jam_list[i]
                            tma_raw = col.text.strip() # Contoh: "40 cm (MT)"
                            
                            # Bersihkan data (ambil angka saja)
                            tma_val = tma_raw.split(' ')[0]
                            
                            # Simpan
                            if tma_val.replace('-','').isdigit():
                                all_data.append({
                                    'tanggal': target_date_str,
                                    'jam': jam,
                                    'pintu_air': nama_pintu,
                                    'tma': int(tma_val),
                                    'status_raw': tma_raw
                                })
            else:
                print(f"⚠️ Tabel tidak ditemukan untuk tanggal {target_date_str}")

        except Exception as e:
            print(f"❌ Error pada tanggal {target_date_str}: {e}")
            # Refresh halaman jika error agar loop berikutnya bisa jalan
            driver.get(url)
            time.sleep(3)

    # Tutup browser
    driver.quit()

    # Simpan ke CSV
    if all_data:
        df = pd.DataFrame(all_data)
        # Gabungkan tanggal dan jam menjadi datetime
        df['waktu_lengkap'] = pd.to_datetime(df['tanggal'] + ' ' + df['jam'] + ':00')
        
        # Simpan
        df.to_csv(OUTPUT_FILE, index=False)
        print(f"✅ Selesai! {len(df)} data tersimpan di {OUTPUT_FILE}")
        print(df.head())
    else:
        print("❌ Tidak ada data yang berhasil diambil.")

if __name__ == "__main__":
    scrape_history()

In [ ]:
import pandas as pd
import time
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# --- KONFIGURASI ---
START_DATE = "2024-01-01"
END_DATE = "2024-01-03" # Coba rentang pendek dulu
OUTPUT_FILE = "../data/data_tma_bpbd_fixed.csv"

def scrape_history_robust():
    # 1. SETUP BROWSER YANG LEBIH STABIL
    options = webdriver.ChromeOptions()
    # Opsi anti-crash
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    # options.add_argument("--headless") # Aktifkan nanti jika sudah lancar
    
    # Hapus log error yang mengganggu
    options.add_argument("--log-level=3") 

    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    except Exception as e:
        print(f"❌ Gagal membuka Chrome. Pastikan Google Chrome terinstall: {e}")
        return

    url = "https://bpbd.jakarta.go.id/waterlevel"
    
    # Buka URL pertama kali
    driver.get(url)
    time.sleep(3) 

    all_data = []
    
    start = datetime.strptime(START_DATE, "%Y-%m-%d")
    end = datetime.strptime(END_DATE, "%Y-%m-%d")
    date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days + 1)]

    wait = WebDriverWait(driver, 15) # Waktu tunggu maksimal 15 detik

    print(f"🔄 Memulai scraping...")

    for date_obj in date_generated:
        target_date_str = date_obj.strftime("%Y-%m-%d")
        print(f"⏳ Sedang memproses: {target_date_str}")
        
        try:
            # 1. Masukkan Tanggal via JavaScript (Lebih aman)
            date_input = wait.until(EC.presence_of_element_located((By.ID, "datetma")))
            driver.execute_script(f"arguments[0].value = '{target_date_str}';", date_input)
            
            # 2. Klik Tombol Filter via JavaScript
            # Menggunakan JS Click mencegah error jika tombol tertutup elemen lain
            submit_btn = driver.find_element(By.CLASS_NAME, "mybutton1")
            driver.execute_script("arguments[0].click();", submit_btn)
            
            # 3. Tunggu Tabel Muncul Kembali
            # Kita tunggu sampai elemen tabel lama hilang atau halaman refresh
            time.sleep(5) # Jeda aman untuk reload halaman
            
            # 4. Ambil HTML
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            table = soup.find('table') # Cari tabel apa saja
            
            if table:
                rows = table.find_all('tr')
                # Cek apakah header tabel sesuai (ada kata 'Pintu Air')
                # Karena struktur tabel BPBD agak unik, kita ambil data baris per baris
                
                # Baris 1: Header (Waktu/Jam)
                if len(rows) > 1:
                    header_cols = rows[1].find_all('td')
                    jam_list = [c.text.strip() for c in header_cols if c.text.strip() != 'Pintu Air']
                    
                    # Baris 2 ke bawah: Data
                    for row in rows[2:]:
                        cols = row.find_all('td')
                        if len(cols) < 2: continue # Skip baris kosong
                        
                        nama_pintu = cols[0].text.strip()
                        
                        # Loop kolom jam
                        for i, col in enumerate(cols[1:]):
                            if i < len(jam_list):
                                jam_val = jam_list[i]
                                tma_text = col.text.strip() # Misal: "40 cm (MT)"
                                
                                # Ambil angka saja
                                tma_clean = tma_text.split(' ')[0]
                                
                                if tma_clean.replace('-','').isdigit():
                                    all_data.append({
                                        'tanggal': target_date_str,
                                        'jam': jam_val,
                                        'lokasi': nama_pintu,
                                        'tma': int(tma_clean),
                                        'raw': tma_text
                                    })
                print(f"   ✅ Berhasil: {len(jam_list)} titik waktu diambil.")
            else:
                print(f"   ⚠️ Tabel tidak muncul/kosong.")

        except Exception as e:
            print(f"   ❌ Error saat proses: {e}")
            # Jika error fatal, coba refresh halaman utama untuk reset
            try:
                driver.get(url)
                time.sleep(3)
            except:
                pass

    driver.quit()

    # Simpan
    if all_data:
        df = pd.DataFrame(all_data)
        df.to_csv(OUTPUT_FILE, index=False)
        print(f"\n🎉 SELESAI! Data tersimpan di: {OUTPUT_FILE}")
        print(df.head())
    else:
        print("\n❌ Tidak ada data yg terambil.")

scrape_history_robust()

In [ ]:
import pandas as pd
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

# --- LIBRARY KHUSUS EDGE ---
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- KONFIGURASI ---
START_DATE = "2024-01-01"
END_DATE = "2024-01-03"  # Coba 3 hari dulu
OUTPUT_FILE = "data_tma_edge.csv"

def scrape_with_edge():
    # 1. SETUP MICROSOFT EDGE
    edge_options = EdgeOptions()
    edge_options.use_chromium = True # Pastikan mode Chromium aktif
    
    # Opsi agar browser stabil dan tidak crash memori
    edge_options.add_argument("--no-sandbox")
    edge_options.add_argument("--disable-dev-shm-usage")
    edge_options.add_argument("--disable-gpu")
    
    # Uncomment baris di bawah ini jika tidak ingin melihat jendela browser terbuka (Background Mode)
    # edge_options.add_argument("--headless") 
    
    # Hapus log "DevTools listening on..." yang mengganggu
    edge_options.add_experimental_option('excludeSwitches', ['enable-logging'])

    print("🚀 Sedang menyiapkan Microsoft Edge Driver...")
    try:
        service = EdgeService(EdgeChromiumDriverManager().install())
        driver = webdriver.Edge(service=service, options=edge_options)
    except Exception as e:
        print(f"❌ Gagal membuka Edge. Error: {e}")
        return

    url = "https://bpbd.jakarta.go.id/waterlevel"
    driver.get(url)
    time.sleep(3) # Tunggu loading awal

    all_data = []
    
    # Buat rentang tanggal
    start = datetime.strptime(START_DATE, "%Y-%m-%d")
    end = datetime.strptime(END_DATE, "%Y-%m-%d")
    date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days + 1)]

    wait = WebDriverWait(driver, 15)

    print(f"🔄 Memulai scraping data historis...")

    for date_obj in date_generated:
        target_date_str = date_obj.strftime("%Y-%m-%d")
        print(f"⏳ Memproses tanggal: {target_date_str}")
        
        try:
            # 2. INJEKSI TANGGAL (JavaScript)
            # Teknik ini paling aman untuk menghindari error ketik
            date_input = wait.until(EC.presence_of_element_located((By.ID, "datetma")))
            driver.execute_script(f"arguments[0].value = '{target_date_str}';", date_input)
            
            # 3. KLIK TOMBOL FILTER
            submit_btn = driver.find_element(By.CLASS_NAME, "mybutton1")
            driver.execute_script("arguments[0].click();", submit_btn)
            
            # 4. TUNGGU RELOAD
            time.sleep(5) # Waktu tunggu aman agar tabel ter-render sempurna
            
            # 5. AMBIL DATA DENGAN BEAUTIFULSOUP
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            table = soup.find('table')
            
            if table:
                rows = table.find_all('tr')
                
                # Cek header untuk memastikan tabel benar
                if len(rows) > 1:
                    # Baris header (jam) biasanya baris ke-2 (indeks 1)
                    header_cols = rows[1].find_all('td')
                    jam_list = [c.text.strip() for c in header_cols if 'Pintu' not in c.text]
                    
                    # Loop data (mulai baris ke-3 / indeks 2)
                    for row in rows[2:]:
                        cols = row.find_all('td')
                        if len(cols) < 2: continue
                        
                        nama_pintu = cols[0].text.strip()
                        
                        # Loop kolom jam
                        for i, col in enumerate(cols[1:]):
                            if i < len(jam_list):
                                jam_val = jam_list[i]
                                tma_raw = col.text.strip() # Contoh: "40 cm (MT)"
                                
                                # Bersihkan angka
                                tma_clean = tma_raw.split(' ')[0]
                                
                                # Pastikan itu angka valid
                                if tma_clean.replace('-','').isdigit():
                                    all_data.append({
                                        'tanggal': target_date_str,
                                        'jam': jam_val,
                                        'lokasi': nama_pintu,
                                        'tma': int(tma_clean),
                                        'kondisi': tma_raw.split('(')[-1].replace(')','') if '(' in tma_raw else ''
                                    })
                print(f"   ✅ Sukses: {len(jam_list)} jam data diambil.")
            else:
                print(f"   ⚠️ Tabel kosong/tidak muncul.")

        except Exception as e:
            print(f"   ❌ Gagal: {e}")
            driver.get(url) # Refresh jika macet
            time.sleep(3)

    driver.quit()

    # Simpan Hasil
    if all_data:
        df = pd.DataFrame(all_data)
        # Gabung tanggal dan jam jadi datetime object
        df['waktu'] = pd.to_datetime(df['tanggal'] + ' ' + df['jam'] + ':00')
        df.to_csv(OUTPUT_FILE, index=False)
        print(f"\n🎉 SELESAI! {len(df)} data tersimpan di '{OUTPUT_FILE}'")
        print(df.head())
    else:
        print("\n❌ Tidak ada data yang berhasil diambil.")

if __name__ == "__main__":
    scrape_with_edge()

🚀 Sedang menyiapkan Microsoft Edge Driver...
❌ Gagal membuka Edge. Error: Could not reach host. Are you offline?


In [3]:
import pandas as pd
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

# === SELENIUM (EDGE) ===
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# ==========================
# KONFIGURASI
# ==========================
START_DATE = "2024-01-01"
END_DATE   = "2024-01-03"   # ubah sesuai kebutuhan
OUTPUT_FILE = "data_tma_bpbd.csv"
URL = "https://bpbd.jakarta.go.id/waterlevel"


# ==========================
# SETUP EDGE DRIVER
# ==========================
def setup_driver():
    edge_options = EdgeOptions()
    edge_options.use_chromium = True

    edge_options.add_argument("--no-sandbox")
    edge_options.add_argument("--disable-dev-shm-usage")
    edge_options.add_argument("--disable-gpu")
    # edge_options.add_argument("--headless")  # aktifkan jika mau background

    edge_options.add_experimental_option(
        'excludeSwitches', ['enable-logging']
    )

    # service = EdgeService(
    #     EdgeChromiumDriverManager().install()
    # )
    
    service = EdgeService(
        executable_path=r"D:\Program_Files\Webdriver\msedgedriver.exe"
    )

    driver = webdriver.Edge(
        service=service,
        options=edge_options
    )
    return driver


# ==========================
# SCRAPING FUNCTION
# ==========================
def scrape_bpbd():
    driver = setup_driver()
    wait = WebDriverWait(driver, 20)

    print("🚀 Membuka halaman BPBD...")
    driver.get(URL)

    all_data = []

    start = datetime.strptime(START_DATE, "%Y-%m-%d")
    end   = datetime.strptime(END_DATE, "%Y-%m-%d")

    date_list = [
        start + timedelta(days=i)
        for i in range((end - start).days + 1)
    ]

    for date_obj in date_list:
        target_date = date_obj.strftime("%Y-%m-%d")
        print(f"\n📅 Memproses tanggal: {target_date}")

        try:
            # === SET TANGGAL (TRIGGER CHANGE EVENT) ===
            date_input = wait.until(
                EC.presence_of_element_located((By.ID, "datetma"))
            )

            driver.execute_script("""
                arguments[0].value = arguments[1];
                arguments[0].dispatchEvent(new Event('change'));
            """, date_input, target_date)

            # === KLIK FILTER ===
            filter_btn = driver.find_element(By.CLASS_NAME, "mybutton1")
            driver.execute_script("arguments[0].click();", filter_btn)

            # === TUNGGU TABEL TERISI ===
            wait.until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "#tabletma tbody tr")
                )
            )

            time.sleep(1)  # buffer kecil agar stabil

            # === PARSE HTML ===
            soup = BeautifulSoup(driver.page_source, "html.parser")

            table = soup.select_one("#tabletma table")
            thead = table.find("thead")
            tbody = table.find("tbody")

            # === AMBIL JAM ===
            header_rows = thead.find_all("tr")
            jam_cells = header_rows[-1].find_all("td")[1:-1]
            jam_list = [j.text.strip() for j in jam_cells]

            print(f"   ⏱️  Jam terdeteksi: {len(jam_list)}")

            # === AMBIL DATA ===
            for row in tbody.find_all("tr"):
                cols = row.find_all("td")
                nama_pintu = cols[0].text.strip()

                for i, col in enumerate(cols[1:]):
                    if i >= len(jam_list):
                        continue

                    tma_raw = col.text.strip()
                    if not tma_raw:
                        continue

                    angka = tma_raw.split()[0]

                    if angka.replace("-", "").isdigit():
                        all_data.append({
                            "tanggal": target_date,
                            "jam": jam_list[i],
                            "lokasi": nama_pintu,
                            "tma_cm": int(angka),
                            "status": (
                                tma_raw.split("(")[-1].replace(")", "")
                                if "(" in tma_raw else ""
                            )
                        })

            print(f"   ✅ Data terkumpul sementara: {len(all_data)} baris")

        except Exception as e:
            print(f"   ❌ ERROR pada {target_date}: {e}")
            driver.get(URL)
            time.sleep(3)

    driver.quit()

    # ==========================
    # SIMPAN HASIL
    # ==========================
    if not all_data:
        print("\n❌ Tidak ada data yang berhasil diambil.")
        return

    df = pd.DataFrame(all_data)

    df["waktu"] = pd.to_datetime(
        df["tanggal"] + " " + df["jam"] + ":00"
    )

    df.to_csv(OUTPUT_FILE, index=False)

    print("\n🎉 SCRAPING SELESAI")
    print(f"📁 File: {OUTPUT_FILE}")
    print(df.head())


# ==========================
# MAIN
# ==========================
if __name__ == "__main__":
    scrape_bpbd()


🚀 Membuka halaman BPBD...


WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: MicrosoftEdge=144.0.3719.82)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff633775ab5
	0x7ff633775b14
	0x7ff633ac2844
	0x7ff6334cb8cc
	0x7ff6334c0d5f
	0x7ff6334c238c
	0x7ff6334c0fcd
	0x7ff6334c0bd0
	0x7ff6334c0935
	0x7ff6334bea31
	0x7ff6334bf0b2
	0x7ff6334d14b8
	0x7ff633552e44
	0x7ff6335375ea
	0x7ff63350d6b7
	0x7ff6335528f7
	0x7ff63350cbcc
	0x7ff63350be26
	0x7ff63350c9f3
	0x7ff6335d0214
	0x7ff6335cc685
	0x7ff6335dce73
	0x7ff633790de8
	0x7ff6337996b6
	0x7ff63377d194
	0x7ff63377d2d9
	0x7ff63376b172
	0x7ffe9163259d
	0x7ffe92c2af78


In [6]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from tqdm import tqdm

# ======================
# KONFIGURASI
# ======================
START_DATE = "2024-01-01"
END_DATE   = "2025-12-31"
OUTPUT_CSV = "bpbd_waterlevel_2tahun.csv"

BASE_URL = "https://103.209.7.34/waterlevel"

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Accept-Language": "id-ID,id;q=0.9,en;q=0.8"
}

# ======================
# HELPER
# ======================
def daterange(start, end):
    start_date = datetime.strptime(start, "%Y-%m-%d")
    end_date   = datetime.strptime(end, "%Y-%m-%d")

    for n in range((end_date - start_date).days + 1):
        yield start_date + timedelta(days=n)

# ======================
# SCRAPER CORE
# ======================
def scrape_bpbd():
    all_rows = []

    for date in tqdm(list(daterange(START_DATE, END_DATE)), desc="Scraping BPBD"):
        date_str = date.strftime("%Y-%m-%d")

        params = {
            "datetma": date_str
        }

        try:
            resp = requests.get(
                BASE_URL,
                headers=HEADERS,
                params=params,
                timeout=30
            )

            if resp.status_code != 200:
                continue

            soup = BeautifulSoup(resp.text, "html.parser")
            table = soup.find("table", id="tabletma")

            if not table:
                continue

            rows = table.find_all("tr")
            if len(rows) < 3:
                continue

            # === JAM (HEADER BARIS KE-2) ===
            header_cells = rows[1].find_all("td")
            jam_list = [
                c.text.strip()
                for c in header_cells
                if ":" in c.text
            ]

            # === DATA (MULAI BARIS KE-3) ===
            for row in rows[2:]:
                cols = row.find_all("td")
                if len(cols) < 2:
                    continue

                pintu_air = cols[0].text.strip()

                for i, cell in enumerate(cols[1:]):
                    if i >= len(jam_list):
                        continue

                    raw = cell.text.strip()
                    if not raw or raw == "-":
                        continue

                    try:
                        tinggi = int(raw.split()[0])
                    except:
                        continue

                    kondisi = ""
                    if "(" in raw:
                        kondisi = raw.split("(")[-1].replace(")", "")

                    all_rows.append({
                        "tanggal": date_str,
                        "jam": jam_list[i],
                        "pintu_air": pintu_air,
                        "tma_cm": tinggi,
                        "kondisi": kondisi
                    })

        except Exception as e:
            print(f"Gagal {date_str}: {e}")

    return pd.DataFrame(all_rows)

# ======================
# MAIN
# ======================
if __name__ == "__main__":
    df = scrape_bpbd()

    if df.empty:
        print("❌ Tidak ada data yang berhasil diambil.")
    else:
        df["waktu"] = pd.to_datetime(
            df["tanggal"] + " " + df["jam"] + ":00"
        )
        df.to_csv(OUTPUT_CSV, index=False)
        print(f"✅ Selesai! {len(df)} baris disimpan ke {OUTPUT_CSV}")
        print(df.head())

Scraping BPBD:   0%|          | 3/731 [00:00<01:10, 10.32it/s]

Gagal 2024-01-01: HTTPSConnectionPool(host='103.209.7.34', port=443): Max retries exceeded with url: /waterlevel?datetma=2024-01-01 (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: IP address mismatch, certificate is not valid for '103.209.7.34'. (_ssl.c:1016)")))
Gagal 2024-01-02: HTTPSConnectionPool(host='103.209.7.34', port=443): Max retries exceeded with url: /waterlevel?datetma=2024-01-02 (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: IP address mismatch, certificate is not valid for '103.209.7.34'. (_ssl.c:1016)")))
Gagal 2024-01-03: HTTPSConnectionPool(host='103.209.7.34', port=443): Max retries exceeded with url: /waterlevel?datetma=2024-01-03 (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: IP address mismatch, certificate is not valid for '103.209.7.34'. (_ssl.c:1016)")))
Gagal 2024-01-04: HTTPSCo

Scraping BPBD:   1%|          | 7/731 [00:00<00:59, 12.11it/s]

Gagal 2024-01-05: HTTPSConnectionPool(host='103.209.7.34', port=443): Max retries exceeded with url: /waterlevel?datetma=2024-01-05 (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: IP address mismatch, certificate is not valid for '103.209.7.34'. (_ssl.c:1016)")))
Gagal 2024-01-06: HTTPSConnectionPool(host='103.209.7.34', port=443): Max retries exceeded with url: /waterlevel?datetma=2024-01-06 (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: IP address mismatch, certificate is not valid for '103.209.7.34'. (_ssl.c:1016)")))
Gagal 2024-01-07: HTTPSConnectionPool(host='103.209.7.34', port=443): Max retries exceeded with url: /waterlevel?datetma=2024-01-07 (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: IP address mismatch, certificate is not valid for '103.209.7.34'. (_ssl.c:1016)")))


Scraping BPBD:   2%|▏         | 11/731 [00:00<00:52, 13.74it/s]

...

Gagal 2025-12-25: HTTPSConnectionPool(host='103.209.7.34', port=443): Max retries exceeded with url: /waterlevel?datetma=2025-12-25 (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: IP address mismatch, certificate is not valid for '103.209.7.34'. (_ssl.c:1016)")))
Gagal 2025-12-26: HTTPSConnectionPool(host='103.209.7.34', port=443): Max retries exceeded with url: /waterlevel?datetma=2025-12-26 (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: IP address mismatch, certificate is not valid for '103.209.7.34'. (_ssl.c:1016)")))
Gagal 2025-12-27: HTTPSConnectionPool(host='103.209.7.34', port=443): Max retries exceeded with url: /waterlevel?datetma=2025-12-27 (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: IP address mismatch, certificate is not valid for '103.209.7.34'. (_ssl.c:1016)")))


Scraping BPBD: 100%|█████████▉| 729/731 [01:02<00:00, 13.03it/s]

Gagal 2025-12-28: HTTPSConnectionPool(host='103.209.7.34', port=443): Max retries exceeded with url: /waterlevel?datetma=2025-12-28 (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: IP address mismatch, certificate is not valid for '103.209.7.34'. (_ssl.c:1016)")))
Gagal 2025-12-29: HTTPSConnectionPool(host='103.209.7.34', port=443): Max retries exceeded with url: /waterlevel?datetma=2025-12-29 (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: IP address mismatch, certificate is not valid for '103.209.7.34'. (_ssl.c:1016)")))
Gagal 2025-12-30: HTTPSConnectionPool(host='103.209.7.34', port=443): Max retries exceeded with url: /waterlevel?datetma=2025-12-30 (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: IP address mismatch, certificate is not valid for '103.209.7.34'. (_ssl.c:1016)")))


Scraping BPBD: 100%|██████████| 731/731 [01:02<00:00, 11.65it/s]

Gagal 2025-12-31: HTTPSConnectionPool(host='103.209.7.34', port=443): Max retries exceeded with url: /waterlevel?datetma=2025-12-31 (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: IP address mismatch, certificate is not valid for '103.209.7.34'. (_ssl.c:1016)")))
❌ Tidak ada data yang berhasil diambil.
